# IN4080: obligatory assignment 1 (Autumn 2024)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, September 15 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

## Part 1 : Tokenisation

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [1]:
from typing import List
import re

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""

    # Implement here your basic tokenisation
    pattern = r"\w+(?:[.,:;!?()'\"`-]\w+)?|[^\w\s]"
    tokens = re.findall(pattern, text)
    return tokens

**Why I Use This Regular Expression Pattern "\w+(?:[.,:;!?()'\"`-]\w+)?|[^\w\s]"**

- For certain compound words, such as **["d'agneau", "Mette-Marit"]**, are treated as single tokens. This means that if a word contains specific punctuation (like an apostrophe or hyphen), it will be recognized as a complete word rather than being split.
- For symbols that have whitespace before and after, they are treated as separate tokens
  - The string **"d'agneau - girolles"** will be tokenized into **["d'agneau", "-", "girolles"]**. Here, the hyphen is recognized as a distinct token due to the spaces surrounding it.
  - Similarly, the string **"Presidenter og prinser | Bak"** will be tokenized into **["Presidenter", "og", "prinser", "|", "Bak"]**. In this case, **the pipe symbol (|)** is treated as a separate token

In [2]:
ex_0 = "Pierre, who works at NR, also teaches at UiO."
ex_1 = "Filet d'agneau - girolles a la creme"
ex_2 = "Mette-Marit hverdagskost"
ex_3 = "Presidenter og prinser | Bak "

result_0 = basic_tokenize(ex_0)
result_1 = basic_tokenize(ex_1)
result_2 = basic_tokenize(ex_2)
result_3 = basic_tokenize(ex_3)

print("Tokenization of \"{}\" is {}".format(ex_0, result_0))
print("Number of tokens:", len(result_0))

print("Tokenization of \"{}\" is {}".format(ex_1, result_1))
print("Number of tokens:", len(result_1))

print("Tokenization of \"{}\" is {}".format(ex_2, result_2))
print("Number of tokens:", len(result_2))

print("Tokenization of \"{}\" is {}".format(ex_3, result_3))
print("Number of tokens:", len(result_3))

Tokenization of "Pierre, who works at NR, also teaches at UiO." is ['Pierre', ',', 'who', 'works', 'at', 'NR', ',', 'also', 'teaches', 'at', 'UiO', '.']
Number of tokens: 12
Tokenization of "Filet d'agneau - girolles a la creme" is ['Filet', "d'agneau", '-', 'girolles', 'a', 'la', 'creme']
Number of tokens: 7
Tokenization of "Mette-Marit hverdagskost" is ['Mette-Marit', 'hverdagskost']
Number of tokens: 2
Tokenization of "Presidenter og prinser | Bak " is ['Presidenter', 'og', 'prinser', '|', 'Bak']
Number of tokens: 5


We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 

In [3]:
# Read the file ndt_test_lm.txt
with open('ndt_test_lm.txt', 'r') as file:
    text = file.read()

# Tokenize the text
result_ndt_test_lm = basic_tokenize(text)

print("Number of tokens:", len(result_ndt_test_lm))
print("Number of types (distinct words):", len(set(result_ndt_test_lm)))

Number of tokens: 255381
Number of types (distinct words): 30835


We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [4]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            # progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                # progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ? 

In [5]:
# Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus
bpe_tokenizer = BPETokenizer('ndt_train_lm.txt')

# Apply this tokenizer on `ndt_test_lm.txt`
with open('ndt_test_lm.txt', 'r') as file:
    text = file.read()

result_ndt_test_lm_bpe = list(bpe_tokenizer(text))

# Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data
print("Number of tokens:", len(result_ndt_test_lm_bpe))
print("Number of types (distinct subwords):", len(set(result_ndt_test_lm_bpe)))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/226096 [00:00<?, ?it/s]

Number of tokens: 386099
Number of types (distinct subwords): 4408


In [6]:
result_bpe_0 = list(bpe_tokenizer(ex_0))
result_bpe_1 = list(bpe_tokenizer(ex_1))
result_bpe_2 = list(bpe_tokenizer(ex_2))
result_bpe_3 = list(bpe_tokenizer(ex_3))

print("Tokenization of \"{}\" by BPE tokenizer is {}".format(ex_0, result_bpe_0))
print("Number of tokens by BPE tokenizer:", len(result_bpe_0))

print("Tokenization of \"{}\" by BPE tokenizer is {}".format(ex_1, result_bpe_1))
print("Number of tokens by BPE tokenizer:", len(result_bpe_1))

print("Tokenization of \"{}\" by BPE method is {}".format(ex_2, result_bpe_2))
print("Number of tokens by BPE tokenizer:", len(result_bpe_2))

print("Tokenization of \"{}\" by BPE method is {}".format(ex_3, result_bpe_3))
print("Number of tokens by BPE tokenizer:", len(result_bpe_3))

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Tokenization of "Pierre, who works at NR, also teaches at UiO." by BPE tokenizer is ['P', 'i', 'er', 're, ', 'w', 'ho ', 'w', 'or', 'ks ', 'at ', 'NR', ', ', 'al', 'so', ' ', 'te', 'a', 'ch', 'es ', 'at ', 'U', 'i', 'O', '. ']
Number of tokens by BPE tokenizer: 24
Tokenization of "Filet d'agneau - girolles a la creme" by BPE tokenizer is ['Fi', 'let ', 'd', "'", 'ag', 'ne', 'a', 'u ', '- ', 'g', 'ir', 'ol', 'les ', 'a ', 'la ', 'c', 're', 'me ']
Number of tokens by BPE tokenizer: 18
Tokenization of "Mette-Marit hverdagskost" by BPE method is ['M', 'ette', '-', 'Mar', 'it ', 'hver', 'dag', 'sk', 'os', 't ']
Number of tokens by BPE tokenizer: 10
Tokenization of "Presidenter og prinser | Bak " by BPE method is ['Presiden', 'ter ', 'og ', 'prin', 'ser ', '| ', 'B', 'ak ']
Number of tokens by BPE tokenizer: 8


How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ?
- The number of tokens obtained by the **BPE tokenizer (386,099)** is significantly higher than the number of tokens obtained by the **basic tokenizer (255,381)**. This is because the BPE is a subword tokenization algorithm, which means that each word can be split into multiple subwords, which increases the total token count.
- The number of types (distinct subwords) obtained by the **BPE tokenizer (4,408)** is much smaller compared to the number of types obtained by the **basic tokenizer (30,835)**. This is because BPE merges frequently occurring subword pairs, leading to fewer unique tokens.

__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

In [7]:
class BPETokenizer_v2:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}

            # [Modify]: Split the letters and punctuation separately (to avoid merging them) in the vocabulary
            # [Modify]: Changing from text.split() to re.findall(r"[\w']+|[`.!,:;?()\"'-]", text)
            for token in re.findall(r"[\w']+|[`.!,:;?()\"'-]", text):
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""
        # We first split into whitespace-separated tokens, and then in subwords
        # [Modify]: Split the letters and punctuation separately in input
        # [Modify]: Changing from text.split() to re.findall(r"[\w']+|[`.!,:;?()\"'-]", text)
        words = re.findall(r"[\w']+|[`.!,:;?()\"'-]", input)
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

In [9]:
bpe_tokenizer_v2 = BPETokenizer_v2('ndt_train_lm.txt')

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

In [10]:
# Apply this tokenizer on `ndt_test_lm.txt`
with open('ndt_test_lm.txt', 'r') as file:
    text = file.read()

result_ndt_test_lm_bpe_v2 = list(bpe_tokenizer_v2(text))

# Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data
print("Number of tokens:", len(result_ndt_test_lm_bpe_v2))
print("Number of types (distinct subwords):", len(set(result_ndt_test_lm_bpe_v2)))

  0%|          | 0/258053 [00:00<?, ?it/s]

Number of tokens: 390542
Number of types (distinct subwords): 4301


__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published earlier this year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

## Part 2: N-gram language models

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [11]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer
 
    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
          mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        for i in tqdm(range(len(tokens))):
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                  alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()


    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [12]:
from collections import defaultdict, Counter

def __init__(self, training_corpus_file: str, tokenizer: BPETokenizer, ngram_size: int = 2, alpha_smoothing: float = 0.1):
    """Initialize the N-gram model with:
    - a file path to a training corpus to estimate the N-gram probabilities
    - an already learned BPE tokenizer
    - an N-gram size
    - a smoothing parameter (Laplace smoothing)"""
    LanguageModel.__init__(self, tokenizer)
    self.ngram_size = ngram_size
    self.alpha_smoothing = alpha_smoothing
    
    # We define a simple backoff distribution (here just a uniform distribution)
    self.default_distrib = {token: 1 / len(tokenizer.vocab) for token in tokenizer.vocab}
    self.ngram_probs = {}

    ngram_dict = {}
    context_counts = {}

    # Read the corpus file
    with open(training_corpus_file, 'r') as fd:
        text = fd.read()
        tokens = list(tokenizer(text))
        for i in range(len(tokens) - ngram_size + 1):
            ngram = tuple(tokens[i:i + ngram_size]) # ngram of size n, ex (w1, w2)      
            context = ngram[:-1] # n-1 first tokens of the ngram
            
            # if the ngram is already in the dictionary, increment the count, otherwise add it
            if ngram in ngram_dict:
                ngram_dict[ngram] += 1 
            else:
                ngram_dict[ngram] = 1
            
            # same for the context
            if context in context_counts:
                context_counts[context] += 1
            else:
                context_counts[context] = 1

    # Calculate probabilities with smoothing
    for ngram, count in ngram_dict.items():
        context = ngram[:-1]  # w_1, ..., w_{n-1}
        token = ngram[-1] # w_n
        context_count = context_counts.get(context, 0)
        
        # Apply smoothing to calculate the probability
        # P(w_n | w_1, ..., w_{n-1}) = (count(w_1, ..., w_n) + alpha) / (count(w_1, ..., w_{n-1}) + alpha * |V|)
        smoothed_prob = (count + self.alpha_smoothing) / (context_count + self.alpha_smoothing * len(tokenizer.vocab))
        if context not in self.ngram_probs:
            self.ngram_probs[context] = {}
        self.ngram_probs[context][token] = smoothed_prob

setattr(NGramLanguageModel, '__init__', __init__)

__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

In [13]:
bpe_tokenizer = BPETokenizer('ndt_train_lm.txt')
ngram_lm = NGramLanguageModel('ndt_train_lm.txt', tokenizer=bpe_tokenizer, ngram_size=2, alpha_smoothing=0.1)

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/872824 [00:00<?, ?it/s]

In [14]:
# Compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`
with open('ndt_test_lm.txt', 'r') as file:
    text = file.read()

perplexity = ngram_lm.get_perplexity(text)
print("Perplexity of the test data:", perplexity)

Tokenising input text:


  0%|          | 0/226096 [00:00<?, ?it/s]

Computing perplexity:


  0%|          | 0/386099 [00:00<?, ?it/s]

Perplexity of the test data: 193.03440869640005


__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

In [15]:
# class InterpolatedNGramLanguageModel(NGramLanguageModel):
#     def __init__(self, train_file, tokenizer, ngram_size=2, alpha_smoothing=0.1, lambdas=None):
#         super().__init__(train_file, tokenizer, ngram_size, alpha_smoothing)
#         self.ngram_models = [NGramLanguageModel(train_file, tokenizer, n, alpha_smoothing) for n in range(1, ngram_size + 1)]
#         if lambdas:
#             assert len(lambdas) == ngram_size, "Length of lambdas must match ngram_size"
#             self.lambdas = lambdas
#         else:
#             self.lambdas = [1.0 / ngram_size] * ngram_size

#     def get_interpolated_prob(self, ngram):
#         tokens = ngram.split()
#         prob = 0.0
#         for i in range(1, self.ngram_size + 1):
#             context = ' '.join(tokens[-i:])
#             prob += self.lambdas[i - 1] * self.ngram_models[i - 1].get_prob(context)
#         return prob

#     def get_prob(self, context):
#         return self.get_interpolated_prob(context)

In [16]:
# # Define different sets of lambda values to test
# lambda_sets = [
#     [0.1, 0.9],  # More weight to bigram
#     [0.5, 0.5],  # Equal weight
#     [0.9, 0.1],  # More weight to unigram
# ]

# # Train and evaluate the model with different lambda sets
# for lambdas in lambda_sets:
#     interpolated_ngram_lm = InterpolatedNGramLanguageModel('ndt_train_lm.txt', tokenizer=bpe_tokenizer, ngram_size=2, alpha_smoothing=0.1, lambdas=lambdas)
    
#     with open('ndt_test_lm.txt', 'r') as file:
#         text = file.read()

#     perplexity = interpolated_ngram_lm.get_perplexity(text)
#     print(f"Perplexity of the test data with lambdas {lambdas}: {perplexity}")

In [ ]:
# reference: https://medium.com/mti-technology/n-gram-language-models-b125b9b62e58

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [17]:
def read_data(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            sentence, label = line.strip().split('\t')
            sentences.append(sentence)
            labels.append(label)
    return sentences, labels

In [18]:
train_file_path = 'ndt_train_class.txt'
train_sentences, train_labels = read_data('ndt_train_class.txt')

In [19]:
def create_features(sentences, tokenizer):
    # N x V matrix
    features = np.zeros((len(sentences), len(tokenizer.vocab)))
    for i, sentence in enumerate(sentences):
        for token in tokenizer(sentence):
            features[i][tokenizer.vocab.index(token)] = 1
    return features

In [20]:
def create_labels(labels):
    # N x 1 vector
    # Nynorsk: `nno` = 1
    # Bokmål: `nob` = 0
    return np.array([1 if label == 'nno' else 0 for label in labels])

In [ ]:
bpe_tokenizer = BPETokenizer('ndt_train_lm.txt')

X_train = create_features(train_sentences, bpe_tokenizer)
y_train = create_labels(train_labels)

__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [ ]:
test_file_path = 'ndt_test_class.txt'
test_sentences, test_labels = read_data(test_file_path)

X_test = create_features(test_sentences, bpe_tokenizer)
y_test = create_labels(test_labels)

X_pred = model.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

accuracy = accuracy_score(y_test, X_pred)
recall = recall_score(y_test, X_pred)
precision = precision_score(y_test, X_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)

Accuracy: 0.9728579481397971
Recall: 0.9624466571834993
Precision: 0.9824021372981764


__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 